In [ ]:
!pip install "uvicorn==0.29.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: uvicorn
    Found existing installation: uvicorn 0.38.0
    Uninstalling uvicorn-0.38.0:
      Successfully uninstalled uvicorn-0.38.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mcp 1.18.0 requires uvicorn>=0.31.1; sys_platform != "emscripten", but you have uvicorn 0.29.0 which is incompatible.
google-adk 1.16.0 requires uvicorn<1.0.0,>=0.34.0, but you have uvicorn 0.29.0 which is incompatible.


In [ ]:
from PIL import Image, ImageEnhance, ImageFilter
import numpy as np
import torch
import cv2
from typing import Dict, Any

def calculate_optimal_size(width, height, max_size=512):
    if width <= max_size and height <= max_size:
        new_width = (width // 8) * 8 or 8
        new_height = (height // 8) * 8 or 8
        return new_width, new_height
    aspect_ratio = width / height
    if width > height:
        new_width = max_size
        new_height = int(max_size / aspect_ratio)
    else:
        new_height = max_size
        new_width = int(max_size * aspect_ratio)
    new_width = (new_width // 8) * 8 or 8
    new_height = (new_height // 8) * 8 or 8
    return new_width, new_height

def preprocess_image(image: Image.Image):
    original_width, original_height = image.size
    optimal_width, optimal_height = calculate_optimal_size(original_width, original_height)
    image = image.convert("RGB")

    # Mild noise reduction
    image_array = np.array(image)
    image_array = cv2.bilateralFilter(image_array, 9, 75, 75)

    # Apply mild CLAHE
    clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(8,8))
    channels = cv2.split(image_array)
    channels = [clahe.apply(ch) for ch in channels]
    image_array = cv2.merge(channels)

    # Conditional auto gamma correction
    mean_val = np.mean(image_array)
    if mean_val < 80 or mean_val > 180:
        gamma = 1.0 / (mean_val / 128 + 1e-6)
        inv_gamma = 1.0 / gamma
        table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in np.arange(256)]).astype("uint8")
        image_array = cv2.LUT(image_array, table)

    image = Image.fromarray(image_array)
    image_resized = image.resize((optimal_width, optimal_height), Image.Resampling.LANCZOS)
    return image_resized, (original_width, original_height)

def generate_depth_map(image: Image.Image, depth_estimator):
    image_resized, original_dims = preprocess_image(image)
    depth = depth_estimator(image_resized)["depth"]

    # Resize depth map
    depth = depth.resize((image_resized.size[0], image_resized.size[1]))
    depth_array = np.array(depth)
    depth_array = cv2.normalize(depth_array, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    # Mild CLAHE and Gaussian blur
    clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(8,8))
    depth_array = clahe.apply(depth_array)
    depth_array = cv2.GaussianBlur(depth_array, (3,3), 0)

    depth = Image.fromarray(depth_array).convert("L")
    return depth, original_dims

def enhance_prompt_advanced(prompt: str, settings: Dict[str, Any]) -> str:
    base_prompt = prompt
    if settings.get('enhanceLighting', True):
        base_prompt += ", natural lighting, warm atmosphere"
    if settings.get('preserveColors', False):
        base_prompt += ", maintaining original color palette, color harmony"
    style = settings.get('style', '').lower()
    if 'modern' in style:
        base_prompt += ", clean lines, contemporary design, sleek furniture"
    elif 'traditional' in style:
        base_prompt += ", classic design, elegant furniture, timeless style"
    elif 'minimalist' in style:
        base_prompt += ", simple design, uncluttered space"
    return base_prompt

def apply_color_correction(image: Image.Image, preserve_colors: bool = False) -> Image.Image:
    img_array = np.array(image)
    color_std = img_array.std()
    if preserve_colors or color_std < 60:
        image = ImageEnhance.Color(image).enhance(1.05)
        image = ImageEnhance.Contrast(image).enhance(1.03)
    else:
        image = ImageEnhance.Color(image).enhance(1.08)
        image = ImageEnhance.Contrast(image).enhance(1.05)
    return image

def upscale_image(image: Image.Image, scale_factor: int = 2) -> Image.Image:
    width, height = image.size
    return image.resize((width * scale_factor, height * scale_factor), Image.Resampling.LANCZOS)

def post_process_image_advanced(image: Image.Image, settings: Dict[str, Any]) -> Image.Image:
    image = image.convert("RGB")
    avg_brightness = np.mean(np.array(image))
    if avg_brightness < 100 or settings.get('enhanceLighting', True):
        factor = 1.2 if avg_brightness < 80 else 1.1
        image = ImageEnhance.Brightness(image).enhance(factor)
    image = apply_color_correction(image, settings.get('preserveColors', False))
    if not settings.get('preserveColors', False):
        image = image.filter(ImageFilter.UnsharpMask(radius=1, percent=60, threshold=5))
    if settings.get('enableUpscaling', False):
        image = upscale_image(image, 2)
    return image

def generate_image_advanced(prompt: str, image: Image.Image, pipe, depth_estimator, settings: Dict[str, Any]):
    depth_map, original_dims = generate_depth_map(image, depth_estimator)
    enhanced_prompt = enhance_prompt_advanced(prompt, settings)
    generation_params = {
        "prompt": enhanced_prompt,
        "image": depth_map,
        "num_inference_steps": settings.get('steps', 20),
        "guidance_scale": min(8.0, settings.get('guidanceScale', 7.5)),
        "strength": min(0.7, settings.get('strength', 0.8)),
        "num_images_per_prompt": 1,
        "negative_prompt": "dark, dim, poorly lit, low quality, blurry, distorted, deformed"
    }
    if settings.get('seed', 0) > 0:
        torch.manual_seed(settings['seed'])
        generation_params["generator"] = torch.Generator().manual_seed(settings['seed'])
    output = pipe(**generation_params).images[0]
    output = post_process_image_advanced(output, settings)
    if not settings.get('enableUpscaling', False):
        output = output.resize(original_dims, Image.Resampling.LANCZOS)
    return output

def generate_multiple_variations(prompt: str, image: Image.Image, pipe, depth_estimator, settings: Dict[str, Any], num_variations: int = 3):
    variations = []
    for i in range(num_variations):
        variation_settings = settings.copy()
        variation_settings['seed'] = settings.get('seed', 0) + i
        variation_settings['guidance_scale'] = min(8.0, settings.get('guidanceScale', 7.5) + i * 0.2)
        variation_settings['strength'] = min(0.7, settings.get('strength', 0.8))
        variation = generate_image_advanced(prompt, image, pipe, depth_estimator, variation_settings)
        variations.append(variation)
    return variations


In [ ]:
import torch
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from transformers import pipeline as transformers_pipeline

# Check if CUDA is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load ControlNet (depth)
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-depth",
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
)

pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    safety_checker=None,  # Disable safety checker to prevent black images
    requires_safety_checker=False
)

# Use a more stable scheduler
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

# Set pipeline to evaluation mode
pipe.unet.eval()
pipe.vae.eval()
if hasattr(pipe, 'text_encoder'):
    pipe.text_encoder.eval()

# Only enable CPU offload if CUDA is available
if device == "cuda":
    pipe.enable_model_cpu_offload()
else:
    # For CPU, move the model to CPU and use optimizations
    pipe = pipe.to(device)
    # Use memory efficient attention for CPU
    pipe.enable_attention_slicing()
    # Use memory efficient xformers if available
    try:
        pipe.enable_xformers_memory_efficient_attention()
        print("Using xformers memory efficient attention")
    except:
        print("xformers not available, using standard attention")
    print("Running on CPU - optimized for speed")

# Depth Estimator
depth_estimator = transformers_pipeline("depth-estimation", device=device)

def get_models():
    return pipe, depth_estimator


Using device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/920 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/1.45G [00:00<?, ?B/s]

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/942 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.37G [00:00<?, ?B/s]

Some weights of DPTForDepthEstimation were not initialized from the model checkpoint at Intel/dpt-large and are newly initialized: ['neck.fusion_stage.layers.0.residual_layer1.convolution1.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution1.weight', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


preprocessor_config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cuda


In [ ]:
!pip install boto3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 5.2 MB/s eta 0:00:00


In [ ]:
import os

# set env vars for this runtime
os.environ["R2_ACCOUNT_ID"] = "bf92e2c165e99c560b2b666664347277"
os.environ["R2_ACCESS_KEY_ID"] = "32114a4eada343584cc871aaf2688677"
os.environ["R2_SECRET_ACCESS_KEY"] = "507e283f740dc3e4347da1ce2533000e2e3a2a60584a252cddb766cbb36fe210"
os.environ["R2_BUCKET_NAME"] = "interior-design-ai"


In [ ]:
import boto3
from botocore.client import Config
from io import BytesIO

R2_ACCOUNT_ID = os.getenv("R2_ACCOUNT_ID")
R2_ACCESS_KEY_ID = os.getenv("R2_ACCESS_KEY_ID")
R2_SECRET_ACCESS_KEY = os.getenv("R2_SECRET_ACCESS_KEY")
R2_BUCKET_NAME = os.getenv("R2_BUCKET_NAME")

r2_endpoint = f"https://{R2_ACCOUNT_ID}.r2.cloudflarestorage.com"

s3_client = boto3.client(
    "s3",
    endpoint_url=r2_endpoint,
    aws_access_key_id=R2_ACCESS_KEY_ID,
    aws_secret_access_key=R2_SECRET_ACCESS_KEY,
    config=Config(signature_version="s3v4"),
    region_name="auto",
)


def upload_to_r2(image: Image.Image, filename: str) -> str:
    buffer = BytesIO()
    image.save(buffer, format="PNG")
    buffer.seek(0)

    s3_client.put_object(
        Bucket=R2_BUCKET_NAME,
        Key=filename,
        Body=buffer,
        ContentType="image/png"
    )

    # return public URL instead of API endpoint
    return f"https://pub-7187c0dd82d349d69a3f7ecd93f5727b.r2.dev/{filename}"


In [ ]:
from fastapi import FastAPI, UploadFile, File, Form, HTTPException
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
from PIL import Image
import uuid
import os
import json
import asyncio
from typing import Optional, Dict, Any
import logging


# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

app = FastAPI(title="Interior Designer AI API", version="2.0.0")

# Add CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Load models
pipe, depth_estimator = get_models()

# Ensure directories exist
os.makedirs("images", exist_ok=True)
os.makedirs("temp", exist_ok=True)

@app.get("/")
async def root():
    return {"message": "Interior Designer AI API", "version": "2.0.0"}

@app.get("/health")
async def health_check():
    return {"status": "healthy", "models_loaded": True}

@app.post("/generate/")
async def generate_basic(
    prompt: str = Form(...),
    file: UploadFile = File(...)
):
    """Basic generation endpoint for backward compatibility"""
    try:
        # Save uploaded image
        input_image = Image.open(file.file).convert("RGB")

        # Default settings
        settings = {
            'steps': 20,
            'guidanceScale': 7.5,
            'strength': 0.8,
            'seed': 0,
            'enableUpscaling': False,
            'preserveColors': False,
            'enhanceLighting': True
        }

        # Generate image
        output_image = generate_image_advanced(prompt, input_image, pipe, depth_estimator, settings)

        # Save and return
        output_path = f"images/{uuid.uuid4()}.png"
        image_url = upload_to_r2(output_image, output_path)

        return JSONResponse({
            "success": True,
            "output": [image_url, image_url],  # Return same image twice for compatibility
            "settings_used": settings
        })

    except Exception as e:
        logger.error(f"Generation error: {str(e)}")
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/generate/advanced")
async def generate_advanced(
    prompt: str = Form(...),
    file: UploadFile = File(...),
    settings: str = Form(default="{}")
):
    """Advanced generation with custom settings"""
    try:
        # Parse settings
        try:
            settings_dict = json.loads(settings)
        except json.JSONDecodeError:
            settings_dict = {}

        # Default settings with overrides
        default_settings = {
            'steps': 20,
            'guidanceScale': 7.5,
            'strength': 0.8,
            'seed': 0,
            'enableUpscaling': False,
            'preserveColors': False,
            'enhanceLighting': True,
            'style': 'Modern',
            'roomType': 'Living Room'
        }
        default_settings.update(settings_dict)

        # Save uploaded image
        input_image = Image.open(file.file).convert("RGB")

        # Log input dimensions
        input_width, input_height = input_image.size
        logger.info(f"Input image dimensions: {input_width}x{input_height}")

        # Generate image
        output_image = generate_image_advanced(prompt, input_image, pipe, depth_estimator, default_settings)

        # Log output dimensions
        output_width, output_height = output_image.size
        logger.info(f"Output image dimensions: {output_width}x{output_height}")

        # Save output
        output_path = f"images/{uuid.uuid4()}.png"
        image_url = upload_to_r2(output_image,output_path)

        return JSONResponse({
            "success": True,
            "output": [image_url, image_url],
            "settings_used": default_settings,
            "input_dimensions": [input_width, input_height],
            "output_dimensions": [output_width, output_height]
        })

    except Exception as e:
        logger.error(f"Advanced generation error: {str(e)}")
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/generate/variations")
async def generate_variations(
    prompt: str = Form(...),
    file: UploadFile = File(...),
    settings: str = Form(default="{}"),
    num_variations: int = Form(default=3)
):
    """Generate multiple variations of the same design"""
    try:
        # Parse settings
        try:
            settings_dict = json.loads(settings)
        except json.JSONDecodeError:
            settings_dict = {}

        # Default settings
        default_settings = {
            'steps': 15,  # Reduced for faster batch processing
            'guidanceScale': 7.5,
            'strength': 0.8,
            'seed': 42,  # Base seed for variations
            'enableUpscaling': False,
            'preserveColors': False,
            'enhanceLighting': True,
            'style': 'Modern',
            'roomType': 'Living Room'
        }
        default_settings.update(settings_dict)

        # Limit variations
        num_variations = min(max(1, num_variations), 5)

        # Save uploaded image
        input_image = Image.open(file.file).convert("RGB")

        # Generate variations
        variations = generate_multiple_variations(
            prompt, input_image, pipe, depth_estimator, default_settings, num_variations
        )

        # Save all variations
        output_paths = []
        for i, variation in enumerate(variations):
            output_path = f"images/{uuid.uuid4()}_var_{i}.png"
            image_url = upload_to_r2(variation,output_path)
            output_paths.append(image_url)

        return JSONResponse({
            "success": True,
            "variations": output_paths,
            "num_generated": len(output_paths),
            "settings_used": default_settings
        })

    except Exception as e:
        logger.error(f"Variations generation error: {str(e)}")
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/generate/batch")
async def generate_batch(
    files: list[UploadFile] = File(...),
    prompt: str = Form(...),
    settings: str = Form(default="{}")
):
    """Batch process multiple images"""
    try:
        # Parse settings
        try:
            settings_dict = json.loads(settings)
        except json.JSONDecodeError:
            settings_dict = {}

        # Default settings optimized for batch processing
        default_settings = {
            'steps': 15,  # Reduced for faster processing
            'guidanceScale': 7.5,
            'strength': 0.8,
            'seed': 0,
            'enableUpscaling': False,
            'preserveColors': False,
            'enhanceLighting': True,
            'style': 'Modern',
            'roomType': 'Living Room'
        }
        default_settings.update(settings_dict)

        # Limit batch size
        files = files[:10]  # Max 10 files

        results = []

        for i, file in enumerate(files):
            try:
                # Process each image
                input_image = Image.open(file.file).convert("RGB")

                # Use different seed for each image
                batch_settings = default_settings.copy()
                batch_settings['seed'] = default_settings['seed'] + i

                output_image = generate_image_advanced(
                    prompt, input_image, pipe, depth_estimator, batch_settings
                )

                # Save output
                output_path = f"images/{uuid.uuid4()}_batch_{i}.png"
                image_url = upload_to_r2(output_image,output_path)

                results.append({
                    "success": True,
                    "original_filename": file.filename,
                    "output_path": image_url,
                    "index": i
                })

            except Exception as e:
                logger.error(f"Batch processing error for file {i}: {str(e)}")
                results.append({
                    "success": False,
                    "original_filename": file.filename,
                    "error": str(e),
                    "index": i
                })

        successful_results = [r for r in results if r["success"]]

        return JSONResponse({
            "success": True,
            "total_processed": len(files),
            "successful": len(successful_results),
            "failed": len(files) - len(successful_results),
            "results": results,
            "settings_used": default_settings
        })

    except Exception as e:
        logger.error(f"Batch processing error: {str(e)}")
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/models/info")
async def get_model_info():
    """Get information about loaded models"""
    return {
        "stable_diffusion_model": "runwayml/stable-diffusion-v1-5",
        "controlnet_model": "lllyasviel/sd-controlnet-depth",
        "depth_estimator": "depth-estimation",
        "device": "cuda" if pipe.device.type == "cuda" else "cpu",
        "optimizations": {
            "memory_efficient_attention": True,
            "cpu_offload": pipe.device.type == "cuda"
        }
    }

@app.delete("/images/{image_id}")
async def delete_image(image_id: str):
    """Delete a generated image from R2"""
    try:
        s3_client.delete_object(Bucket=R2_BUCKET_NAME, Key=image_id)
        return {"success": True, "message": "Image deleted"}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


# uvicorn.run(app, host="0.0.0.0", port=8000)

In [ ]:


!uvicorn main:app --host 0.0.0.0 --port 8000


ERROR:    Error loading ASGI app. Could not import module "main".


In [ ]:
import threading
import uvicorn

def run_app():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Start the server in a daemon thread
thread = threading.Thread(target=run_app, daemon=True)
thread.start()
print("✅ FastAPI server running in background")


✅ FastAPI server running in background


In [ ]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb


INFO:     Started server process [545]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Selecting previously unselected package cloudflared.
(Reading database ... 126675 files and directories currently installed.)
Preparing to unpack cloudflared-linux-amd64.deb ...
Unpacking cloudflared (2025.10.0) ...
Setting up cloudflared (2025.10.0) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
!cloudflared tunnel --url http://0.0.0.0:8000 --no-autoupdate


2025-10-23T11:25:49Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-10-23T11:25:49Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-10-23T11:25:52Z INF +--------------------------------------------------------------------------------------------+
2025-10-23T11:25:52Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-10-23T11:25:52Z INF |  https://determine-graham-environmental-register.trycl

  0%|          | 0/20 [00:00<?, ?it/s]

INFO:     117.213.202.150:0 - "POST /generate/ HTTP/1.1" 200 OK


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

...

INFO:     117.213.202.150:0 - "POST /generate/variations HTTP/1.1" 200 OK


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

INFO:     117.213.202.150:0 - "POST /generate/variations HTTP/1.1" 200 OK


Couldn't connect to the Hub: 404 Client Error. (Request ID: Root=1-68db62cb-2e430c763b5c179f0454be57;a4892e11-54de-4c55-bb1c-791399718277)

Repository Not Found for url: https://huggingface.co/api/models/stabilityai/Stable3D.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated. For more details, see https://huggingface.co/docs/huggingface_hub/authentication.
Will try to load from local cache.


OSError: Cannot load model stabilityai/Stable3D: model is not cached locally and an error occurred while trying to fetch metadata from the Hub. Please check out the root cause in the stacktrace above.